In [18]:
import pandas as pd
import numpy as np
import math

In [11]:
def euclidean_distance(xstart, ystart, xend, yend):
    a = np.array([xstart, ystart])
    b = np.array([xend, yend])
    return np.linalg.norm(a-b)

In [36]:
# read scenario pedestrian statistics
scenario_OSM_rimea6 = "../EX2/task1/output/rimea6_2021-11-08_10-03-44.360/postvis.traj"
scenario_SFM_rimea6 = "../EX2/task2/task2_SFM/output/rimea6_2021-11-08_10-20-14.487/postvis.traj"
scenario_GNM_rimea6 = "../EX2/task2/task2_GNM/output/rimea6_2021-11-08_10-47-35.351/postvis.traj"
df = pd.read_csv(scenario_SFM_rimea6, sep=' ')
df

,pedestrianId,simTime,endTime-PID1,startX-PID1,startY-PID1,endX-PID1,endY-PID1,targetId-PID2
0,1,0.0,0.1,0.701000,2.094803,0.726497,2.095820,5
1,2,0.0,0.1,1.103000,2.094803,1.116464,2.095736,5
2,3,0.0,0.1,0.701000,2.496803,0.724721,2.498429,5
3,4,0.0,0.1,1.103000,2.496803,1.121833,2.499417,5
4,5,0.0,0.1,0.701000,2.898803,0.725747,2.900855,5
...,...,...,...,...,...,...,...,...
2157,18,24.7,24.8,7.707209,8.666910,7.706837,8.722054,5
2158,18,24.8,24.9,7.706837,8.722054,7.706467,8.775714,5
2159,18,24.9,25.0,7.706467,8.775714,7.706101,8.827297,5
2160,18,25.0,25.1,7.706101,8.827297,7.705738,8.876015,5


In [40]:
# create dataframe for storing pedestrians velocities
data = {'pedestrianId':[], 'average_speed':[]}
for p in df['pedestrianId'].unique():
    temp = df[df['pedestrianId'] == p]
    travelled_space = 0
    time_taken = 0
    for _, r in temp.iterrows():
        travelled_space += euclidean_distance(r['startX-PID1'], r['startY-PID1'], r['endX-PID1'], r['endY-PID1'])
        time_taken += r['endTime-PID1'] - r['simTime']  # useless, could be just measure once and multiply per # iter
    data['pedestrianId'].append(p)
    data['average_speed'].append(travelled_space / time_taken if time_taken != 0 else 0)
pedestrian_df = pd.DataFrame(data)
pedestrian_df

# add pedestrian variance in velocities
variance_list = []
std_dev_list = []
for p in df['pedestrianId'].unique():
    temp = df[df['pedestrianId'] == p]
    ped_avg_speed = float(pedestrian_df[pedestrian_df['pedestrianId'] == p]['average_speed'])
    variance_accum = 0
    for _, r in temp.iterrows():
        travelled_space = euclidean_distance(r['startX-PID1'], r['startY-PID1'], r['endX-PID1'], r['endY-PID1'])
        time_taken = r['endTime-PID1'] - r['simTime']  # useless, could be just measure once and multiply per # iter
        variance_accum += ((travelled_space/time_taken - ped_avg_speed))**2
    variance_list.append(variance_accum/len(temp))
    std_dev_list.append(math.sqrt(variance_list[-1]))

pedestrian_df['variance'] = variance_list
pedestrian_df['std_dev'] = std_dev_list
pedestrian_df
        

,pedestrianId,average_speed,variance,std_dev
0,1,1.086695,0.051130,0.226118
1,2,1.062604,0.046868,0.216490
2,3,1.019919,0.076171,0.275992
3,4,1.519372,0.063086,0.251169
4,5,1.303660,0.087612,0.295993
5,6,1.625540,0.089549,0.299247
6,7,1.492314,0.090223,0.300371
7,8,1.294587,0.062285,0.249570
8,9,1.750456,0.117822,0.343251
9,10,1.043054,0.050235,0.224131


In [42]:
pedestrian_df.describe()

,pedestrianId,average_speed,variance,std_dev
count,20.00000,20.000000,20.000000,20.000000
mean,10.50000,1.221708,0.076093,0.269668
std,5.91608,0.343576,0.031274,0.059580
min,1.00000,0.500956,0.021057,0.145112
25%,5.75000,1.017261,0.058317,0.241329
50%,10.50000,1.195393,0.073505,0.271073
75%,15.25000,1.502871,0.089717,0.299528
max,20.00000,1.776543,0.143687,0.379060
